In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/infer_data/train.parquet')
df.ts = (df.ts//1000).astype('int32')
mn = df.ts.min()
df = df.loc[df.ts >= mn+60*60*24*7]

df = df.loc[df['type']!=0]

df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(16614516, 6)


,session,aid,ts,type,hour,day
180,0,974651,1661336218,1,37018,555418
182,0,974651,1661336225,1,37025,555425
183,0,974651,1661336252,1,37052,555452
184,0,280978,1661336260,1,37060,555460
185,0,1521766,1661336265,1,37065,555465


In [3]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio','count_item','count_user','hour_mean','day_mean']

In [4]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean
aid,,,,,
1,1.000000,1,1,56602.000000,575002.000000
3,1.169643,224,157,52163.366071,289763.366071
4,1.000000,8,8,58075.250000,241675.250000
8,1.000000,1,1,49243.000000,567643.000000
10,1.333333,3,2,65387.666667,295787.666667


In [5]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std','hour_std','day_std']

f32 = ['buy_ratio_std','hour_std','day_std']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [6]:
item_features2.head()

,buy_ratio_std,hour_std,day_std
aid,,,
1407951,0.48795,11771.460938,126465.968750
585446,-1.00000,-1.000000,-1.000000
482199,0.00000,20073.347656,203355.421875
427796,0.00000,18306.558594,232731.093750
1692898,0.00000,6665.241211,128518.210938


In [7]:
item_features['repeat'] = item_features.count_item / item_features.count_user

f32 = ['buy_ratio','hour_mean','day_mean','repeat']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item','count_user']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [8]:
item_features = cudf.concat([item_features,item_features2],axis=1)

In [9]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
aid,,,,,,,,,
1,1.000000,1,1,56602.000000,575002.00000,1.000000,-1.000000,-1.000000,-1.000000
3,1.169643,224,157,52163.367188,289763.37500,1.426752,0.376159,15203.037109,144851.250000
4,1.000000,8,8,58075.250000,241675.25000,1.000000,0.000000,15992.715820,135522.828125
8,1.000000,1,1,49243.000000,567643.00000,1.000000,-1.000000,-1.000000,-1.000000
10,1.333333,3,2,65387.667969,295787.65625,1.500000,0.577350,4047.108643,53823.906250


In [10]:
item_features.dtypes

buy_ratio        float32
count_item         int32
count_user         int32
hour_mean        float32
day_mean         float32
repeat           float32
buy_ratio_std    float32
hour_std         float32
day_std          float32
dtype: object

In [11]:
COLS = [f'{x}11' for x in item_features.columns]
item_features.columns = COLS
print(COLS)

['buy_ratio11', 'count_item11', 'count_user11', 'hour_mean11', 'day_mean11', 'repeat11', 'buy_ratio_std11', 'hour_std11', 'day_std11']


In [12]:
item_features.to_parquet('../../data/item_user_features/item40_LB.pqt')